In [1]:
import tensorflow as tf
import time
import os
import numpy as np
import cPickle as cp #serializing and de-serializing a Python object structure
from sklearn import cross_validation
from sliding_window import sliding_window

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# run environment

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.4)
config = tf.ConfigProto(gpu_options=gpu_options)

# params setup

In [3]:
NB_SENSOR_CHANNELS = 120

NUM_CLASSES = 7

# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 24

# Length of the input sequence after convolutional operations
FINAL_SEQUENCE_LENGTH = 8

# Hardcoded step of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_STEP = 12

# Batch Size
BATCH_SIZE = 100

# Number filters convolutional layers
NUM_FILTERS = 64

# Size filters convolutional layers
FILTER_SIZE = 5

# Number of unit in the long short-term recurrent layers
NUM_UNITS_LSTM = 128

# load data

In [4]:
def load_X(X_path):
    file = open(X_path, 'r')
    # Read dataset from disk, dealing with text files' syntax
    X_signal = [np.array(item, dtype=np.float32) for item in [
               line.strip().split('\t') for line in file]]
    file.close()
    return np.array(X_signal)


# Load "y" (the neural network's training and testing outputs)
def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array([elem for elem in [line.strip().split('\t') for line in file]], 
                  dtype=np.int32)
    file.close()
    # Substract 1 to each output class for friendly 0-based indexing
    return y_ 

labels = ['info', 'crit', 'err', 'notice', 'warning', 'alert', 'emerg']

dataset_path = "data/"
print("\n" + "Dataset is now located at: " + dataset_path)

X_train_signals_path = dataset_path + "msg_token_train.txt"
X_test_signals_path = dataset_path + "msg_token_test.txt"
X_train = load_X(X_train_signals_path)
X_test = load_X(X_test_signals_path)

y_train_path = dataset_path + "msg_label_train.txt"
y_test_path = dataset_path + "msg_label_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)

# Separate our training data into test and training.
print("Separating data into 67% training set & 33% test set...")
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    #X_train, y_train, test_size=0.33, random_state=33) #add random state here...
print("Dataset separated.\n")

print(X_train.shape, y_train.shape, y_test.shape)


Dataset is now located at: data/
Separating data into 67% training set & 33% test set...
Dataset separated.

((8360, 120), (8360, 1), (1671, 1))


# sliding window

In [5]:
assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,(ws,1),(ss,1))])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.int32)

# Sensor data is segmented using a sliding window mechanism
y_test
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, 
                                    SLIDING_WINDOW_STEP)
print("after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, 
                                                                          y_test.shape))
# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS, 1))

# Sensor data is segmented using a sliding window mechanism
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, 
                                      SLIDING_WINDOW_STEP)
print("after sliding window (testing): inputs {0}, targets {1}".format(
                                      X_test.shape, y_test.shape))
# Data is reshaped since the input of the network is a 4 dimension tensor
X_train = X_train.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS, 1))

after sliding window (testing): inputs (138, 24, 120), targets (138,)
after sliding window (testing): inputs (138, 24, 120, 1), targets (138,)


# one hot

In [6]:
def one_hot(label):
    """convert label from dense to one hot
      argument:
        label: ndarray dense label ,shape: [sample_num,1]
      return:
        one_hot_label: ndarray  one hot, shape: [sample_num,n_class]
    """
    label_num = len(label)
    new_label = label.reshape(label_num)  # shape : [sample_num]
    # because max is 6, and we will create 7 columns
    n_values = np.max(new_label) + 1
    return np.eye(n_values)[np.array(new_label, dtype=np.int32)]

y_test=one_hot(y_test)
y_train=one_hot(y_train)

print("data is ready")

data is ready


# define network 

In [7]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, w, w2, w3, w4, rnnW, rnnB, lstm_size):
    # l1a shape=(?, 28, 28, 32)
    l1a = tf.nn.relu(tf.nn.conv2d(X, w, strides=[1, 1, 1, 1], padding='VALID'))
    # l1 shape=(?, 14, 14, 32)
    #|l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #l1 = tf.nn.dropout(l1, p_keep_conv)
                     
    # l2a shape=(?, 14, 14, 64)
    l2a = tf.nn.relu(tf.nn.conv2d(l1a, w2, strides=[1, 1, 1, 1], padding='VALID'))
    # l2 shape=(?, 7, 7, 64)
    #l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #l2 = tf.nn.dropout(l2, p_keep_conv)

    # l3a shape=(?, 7, 7, 128)
    l3a = tf.nn.relu(tf.nn.conv2d(l2a, w3, strides=[1, 1, 1, 1], padding='VALID'))        
    # l3 shape=(?, 4, 4, 128)                           
    #l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    # reshape to (?, 2048)
    #l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])  
    #l3 = tf.nn.dropout(l3, p_keep_conv)
                 
    # l3a shape=(?, 7, 7, 128)
    l4a = tf.nn.relu(tf.nn.conv2d(l3a, w4, strides=[1, 1, 1, 1], padding='VALID'))
                     
    shuff = tf.transpose(l4a, [1, 0, 2, 3])
    shp1 = tf.reshape(shuff, [-1, lstm_size])
    # split them to time_step_size (28 arrays)
    X_split = tf.split(shp1, 480, 0) 
    
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size, 
                                        forget_bias=1.0, state_is_tuple=True)  
    lstm_cell2 = tf.contrib.rnn.BasicLSTMCell(lstm_size, 
                                        forget_bias=1.0, state_is_tuple=True)
    # Stack two LSTM layers, both layers has the same shape
    lstm_layers = tf.contrib.rnn.MultiRNNCell([lstm_cell, lstm_cell2], 
                                              state_is_tuple=True)
                     
    outputs, _states = tf.contrib.rnn.static_rnn(lstm_layers, X_split, dtype=tf.float32)
    
    print("tf net end")

    return tf.matmul(outputs[-1], rnnW) + rnnB

# model config

In [8]:
X = tf.placeholder("float", [None, 24, 120, 1])
Y = tf.placeholder("float", [None, 7])

lstm_size = 128
w = init_weights([5, 1, 1, 64])       # 3x3x1 conv, 32 outputs
w2 = init_weights([5, 1, 64, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([5, 1, 64, 64])    # 3x3x32 conv, 128 outputs
w4 = init_weights([5, 1, 64, 64]) # FC 128 * 4 * 4 inputs, 625 outputs

rnnW = init_weights([lstm_size, 7])
rnnB = init_weights([7])
pre_Y = model(X, w, w2, w3, w4, rnnW, rnnB,lstm_size)
print ("get cnn output");

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pre_Y, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(pre_Y, 1)

print("net work done")

tf net end
get cnn output
net work done


# train and test

In [9]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

# Launch the graph in a session
with tf.Session(config=config) as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()

    for i in range(100):
        for start, end in zip(range(0, len(X_train), BATCH_SIZE), 
                              range(BATCH_SIZE, len(X_train)+1, BATCH_SIZE)):
            sess.run(train_op, feed_dict={X: X_train[start:end], Y: y_train[start:end]})

        test_indices = np.arange(len(X_test))  # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:]

        print(i, np.mean(np.argmax(y_test[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: X_test[test_indices]})))

(0, 0.072463768115942032)
(1, 0.41304347826086957)
(2, 0.38405797101449274)
(3, 0.48550724637681159)
(4, 0.40579710144927539)
(5, 0.40579710144927539)
(6, 0.40579710144927539)
(7, 0.43478260869565216)
(8, 0.40579710144927539)
(9, 0.40579710144927539)
(10, 0.40579710144927539)
(11, 0.40579710144927539)
(12, 0.40579710144927539)
(13, 0.40579710144927539)
(14, 0.40579710144927539)
(15, 0.40579710144927539)
(16, 0.40579710144927539)
(17, 0.40579710144927539)
(18, 0.40579710144927539)
(19, 0.40579710144927539)
(20, 0.40579710144927539)
(21, 0.40579710144927539)
(22, 0.40579710144927539)
(23, 0.40579710144927539)
(24, 0.40579710144927539)
(25, 0.40579710144927539)
(26, 0.40579710144927539)
(27, 0.40579710144927539)
(28, 0.40579710144927539)
(29, 0.40579710144927539)
(30, 0.40579710144927539)
(31, 0.40579710144927539)
(32, 0.40579710144927539)
(33, 0.40579710144927539)
(34, 0.40579710144927539)
(35, 0.40579710144927539)
(36, 0.40579710144927539)
(37, 0.40579710144927539)
(38, 0.40579710144927